## Getting started
- Download metadata tables from https://www.kaggle.com/datasets/kaggle/meta-kaggle at /fsx/loubna/kaggle_data/metadata_kaggle/
- Download notebooks dataset from https://www.kaggle.com/datasets/kaggle/meta-kaggle-code at /fsx/loubna/kaggle_data/kaggle-code-data/data (note: it can take many hours)

Some mapping between the code dataset `meta-kaggle-code` and the csv tables available at `meta-kaggle` needs to be done to retrieve the metadata of each notebook, in particular we want to find the dataset name (owner/data_name) to download datasets using kaggle API so we can add information about the dataset used in each notebooks. We also want to add upvotes, title, data description and competition description/title and any other relevant information...

In [ ]:
import pandas as pd
import numpy as np
import json
from pathlib import Path


# the Meta Kaggle Code dataset with notebooks
code_base_path = Path('/fsx/loubna/kaggle_data/kaggle-code-data/data')
# match id to the filename in Meta Kaggle Code
kv_csv = '/fsx/loubna/kaggle_data/metadata_kaggle/KernelVersions.csv'

# to get the name of the dataset used in each notebook
kernelversions_datasetsources_csv = '/fsx/loubna/kaggle_data/metadata_kaggle/KernelVersionDatasetSources.csv'
datasets_versions_csv = '/fsx/loubna/kaggle_data/metadata_kaggle/DatasetVersions.csv'
# to get the org user id of the dataset
datasets_csv = '/fsx/loubna/kaggle_data/metadata_kaggle/Datasets.csv'
# to get owner name from its id
users_csv = '/fsx/loubna/kaggle_data/metadata_kaggle/Users.csv'

In [9]:
# retrieve notebooks, tehre are also .py and .r files to be analyzed
notebooks = code_base_path.glob('*/*/*.ipynb')

In [ ]:
kversions = pd.read_csv(kv_csv)
datasets_versions = pd.read_csv(datasets_versions_csv)
datasets = pd.read_csv(datasets_csv)
kernelversions_datasetsources = pd.read_csv(kernelversions_datasetsources_csv)
users = pd.read_csv(users_csv)

In [16]:
kversions.head()

,Id,ScriptId,ParentScriptVersionId,ScriptLanguageId,AuthorUserId,CreationDate,VersionNumber,Title,EvaluationDate,IsChange,TotalLines,LinesInsertedFromPrevious,LinesChangedFromPrevious,LinesUnchangedFromPrevious,LinesInsertedFromFork,LinesDeletedFromFork,LinesChangedFromFork,LinesUnchangedFromFork,TotalVotes
0,3107,865,NaN,2,294199,04/26/2015 08:03:20,NaN,Keras deep net starter code,04/26/2015,False,158.0,NaN,NaN,NaN,0.0,0.0,0.0,158.0,0
1,3321,991,NaN,1,28963,04/27/2015 15:13:37,NaN,"Yo buddies, let's party",04/27/2015,False,4.0,0.0,0.0,4.0,NaN,NaN,NaN,NaN,0
2,10018,4583,NaN,2,216445,05/27/2015 19:12:59,NaN,scTryOut,05/27/2015,True,81.0,9.0,5.0,67.0,NaN,NaN,NaN,NaN,0
3,10115,4682,NaN,1,1889,05/28/2015 03:39:58,NaN,"Max(Time Elapsed,Mean Time) Benchmark",05/28/2015,True,22.0,0.0,2.0,20.0,0.0,0.0,2.0,20.0,0
4,10194,4702,NaN,2,303928,05/28/2015 11:30:32,NaN,Simple Lasagne NN,05/28/2015,False,246.0,0.0,0.0,247.0,0.0,0.0,2.0,245.0,0


In [21]:
kernelversions_datasetsources.head()

,Id,KernelVersionId,SourceDatasetVersionId
0,292938,888680,1491
1,299142,888506,1491
2,293256,889995,1491
3,293955,889967,1491
4,303817,889848,1491


In [36]:
fp = next(notebooks) 
with open(fp,'r') as f:
    content = f.readlines()[0]
    content = json.loads(content)
    cells = content['cells']
    sample = {"content": cells}
file_id = str(fp).split('/')[-1].split('.')[0]
print(fp)
# the file id is its name
print(file_id)

/fsx/loubna/kaggle_data/kaggle-code-data/data/0069/046/69046416.ipynb
69046416


In [ ]:
from pprint import pprint

pprint(cells)

We now have the content of the notebook, let's retrieve its metadata: dataset, description, competition upvotes..

There are several tables to retrieve this data from:
- KernelVersions for some metadata that will link to other tables
- KernelVersionDatasetSources for the data source
- DatasetVersion with dataset name
- Datasets with owners of the dataset

=> can be used to download the corresponding kaggla dataset

In [38]:
# first metadata
kversion = kversions[kversions['Id']==int(file_id)]
kversion

,Id,ScriptId,ParentScriptVersionId,ScriptLanguageId,AuthorUserId,CreationDate,VersionNumber,Title,EvaluationDate,IsChange,TotalLines,LinesInsertedFromPrevious,LinesChangedFromPrevious,LinesUnchangedFromPrevious,LinesInsertedFromFork,LinesDeletedFromFork,LinesChangedFromFork,LinesUnchangedFromFork,TotalVotes
5221710,69046416,18825679,NaN,9,7571614,07/26/2021 08:39:57,4.0,King County Houses Neighborhood Classification,07/26/2021,True,269.0,10.0,0.0,259.0,NaN,NaN,NaN,NaN,1


In [41]:
# find data source
data_source_kernel = kernelversions_datasetsources[kernelversions_datasetsources['KernelVersionId']==int(file_id)]
data_source_kernel

,Id,KernelVersionId,SourceDatasetVersionId
3297208,91775956,69046416,270


In [45]:
datasets_versions.head(2)

,Id,DatasetId,DatasourceVersionId,CreatorUserId,LicenseName,CreationDate,VersionNumber,Title,Slug,Subtitle,Description,VersionNotes,TotalCompressedBytes,TotalUncompressedBytes
0,6,6,6,1,CC0: Public Domain,07/18/2015 00:51:12,1.0,2013 American Community Survey,2013-american-community-survey,Find insights in the 2013 American Community S...,The [American Community Survey](http://www.cen...,Initial Release,NaN,NaN
1,8,8,8,1,CC0: Public Domain,08/18/2015 21:53:00,1.0,Ocean Ship Logbooks (1750-1850),climate-data-from-ocean-ships,Explore changing climatology with data from ea...,"In the mid-eighteenth to nineteenth centuries,...",Initial release,NaN,NaN


In [54]:
source_id = data_source_kernel['SourceDatasetVersionId']
dataset_name = datasets_versions[datasets_versions['Id']==int(source_id)]
dataset_name

,Id,DatasetId,DatasourceVersionId,CreatorUserId,LicenseName,CreationDate,VersionNumber,Title,Slug,Subtitle,Description,VersionNotes,TotalCompressedBytes,TotalUncompressedBytes
94,270,128,270,680332,CC0: Public Domain,08/25/2016 15:52:49,1.0,"House Sales in King County, USA",housesalesprediction,Predict house price using regression,This dataset contains house sale prices for Ki...,Initial release,2515206.0,2515206.0


In [77]:
# get str in dataset_name["Slug"]
data_name = dataset_name["Slug"].values[0]
data_name

'housesalesprediction'

In [65]:
# check in datasets if a row has id =dataset_name["DatasetId"]
owner = datasets[datasets['Id']==int(dataset_name["DatasetId"])]
owner

,Id,CreatorUserId,OwnerUserId,OwnerOrganizationId,CurrentDatasetVersionId,CurrentDatasourceVersionId,ForumId,Type,CreationDate,LastActivityDate,TotalViews,TotalDownloads,TotalVotes,TotalKernels
11,128,680332,680332.0,NaN,270.0,270.0,1447,2,08/25/2016 15:52:49,02/06/2018,996866,172516,2041,1225


In [66]:
users.head(2)

,Id,UserName,DisplayName,RegisterDate,PerformanceTier
0,1,kaggleteam,Kaggle Team,03/24/2011,5
1,368,antgoldbloom,Anthony Goldbloom,01/20/2010,2


In [68]:
owner["OwnerUserId"]

11    680332.0
Name: OwnerUserId, dtype: float64

In [69]:
# let's get username
user_id = users[users['Id']==int(owner["OwnerUserId"])]
user_id

,Id,UserName,DisplayName,RegisterDate,PerformanceTier
597836,680332,harlfoxem,harlfoxem,08/05/2016,1


In [81]:
final_data = f'{user_id["UserName"].values[0]}/{data_name}'
final_data

'harlfoxem/housesalesprediction'

We can then retrieve the data from: https://www.kaggle.com/datasets/harlfoxem/housesalesprediction 🎉